In [1]:
import numpy as np
import keras as k
import rules
import arena as arn
from bot_base import BOT
import bot_anneal

bot_rd = arn.bot_rd
bot_me = arn.bot_me


In [8]:
def bot_mc_f(basebot, verbos=2, RECORD=True, MAXWIDTH=3, NSIMS_FACTOR=50, NSIMS_MAX=1000):
    return mc.BOT(basebot=basebot, verbos=verbos, RECORD=RECORD, MAXWIDTH=MAXWIDTH, NSIMS_FACTOR=NSIMS_FACTOR, NSIMS_MAX=NSIMS_MAX)

In [3]:
# model_0 = k.models.load_model("model_base/m3841.keras")
model_0 = k.models.load_model("model_base_allrecords/m17544.keras")
model = k.models.load_model("model_small_kernel/m1243.keras")

In [4]:
def ft(N, ARENA):
    s1, s2 = 0, 0
    for i in range(N):
        cards = rules.CARDS.copy()
        np.random.shuffle(cards)
        
        arena = ARENA(verbos=0, cards=cards.copy())
        arena.registerbot([BOT(model), BOT(model_0), BOT(model_0)])
        # arena.registerbot([BOT(model), bot_rd, bot_rd])
        arena.wholegame()
        s1 += (arena.winner == 0)
        
        arena2 = ARENA(verbos=0, cards=cards.copy())
        arena2.registerbot([BOT(model_0), BOT(model), BOT(model)])
        arena2.wholegame()
        s2 += (arena2.winner == 0)
        
    # for i in range(N):
    #     arena = ARENA(verbos=0)
    #     arena.registerbot([BOT(model_0), BOT(model), BOT(model)])
    #     # arena.registerbot([BOT(model_0), bot_rd, bot_rd])
    #     arena.wholegame()
    #     s2 += (arena.winner == 0)
    # return s1
    return s1, s2

In [ ]:
ft(100, arn.ARENA)

In [6]:
import re

def repl(matched):
    d = {"11": "J", "12": "Q", "13": "K", "14": "A", "15": "2", "16": "小王", "17": "大王"}
    return d[matched[0]]

def arenabyscratch(l):
    cards = [rules.str2vec(re.sub(r'1[1-7]', repl, v)) for v in re.split(",", l)]
        
    arena=object.__new__(arn.ARENA)
    arena.verbos = 1
    arena.init = np.array(cards, int)
    arena.remain = arena.init.copy()
    arena.lastplay = np.zeros((3, 15), int)
    arena.pos = 0
    arena.b1 = 2
    arena.b2 = 1
    arena.round = 0
    arena.bot = []
    arena.gameover = False
    arena.winner = None
    arena.getdata()
    arena.RECORD = True
    arena.records = []
    if arena.RECORD:
        arena.records.append(arena.copy())
    return arena

arena = arenabyscratch("3 4 4 4 4 5 5 7 8 9 9 9 10 11 12 13 13 14 14 17, 3 5 5 6 6 6 7 7 7 8 9 10 11 12 13 13 15, 3 3 6 8 8 10 10 11 11 12 12 14 14 15 15 15 16")

In [7]:
# %%time
# bot_hyb = hyb.BOT(bot, bot, verbos=1, NINFER=10, tao=0.04)

arena = arn.ARENA(1, True)
arena.registerbot([bot_anneal.BOT(model,verbos=1, temperature=1), bot_anneal.BOT(model,verbos=1, temperature=1), bot_anneal.BOT(model,verbos=1, temperature=1)])
# arena.registerbot([BOT(model,verbos=1), bot_rd, bot_rd])
arena.wholegame()

0.48217815 7 7 8 8 9 9 10 10
0.44340256 7 7 8 8 9 9
0.43431252 8 8 9 9 10 10
0.3599209 3 4 5 6 7 8 9 10
0.35787514 3 4 5 6 7 8 9 10 J
 0 :  地主 : 4 5 6 7 8 9 10 J | 3 6 7 8 9 10 Q Q Q A A 2
0.76215434 
 1 : 农民1 :  | 3 3 4 5 5 6 6 7 7 10 J J K K K 2 小王
0.74946976 
 2 : 农民2 :  | 3 4 4 5 8 8 9 9 10 J Q K A A 2 2 大王
0.35601988 6 7 8 9 10
0.2758113 3
0.24789901 8
0.2339555 9
0.23015374 10
 3 :  地主 : 6 7 8 9 10 | 3 Q Q Q A A 2
0.65531814 
 4 : 农民1 :  | 3 3 4 5 5 6 6 7 7 10 J J K K K 2 小王
0.8016203 10 J Q K A
0.78625715 9 10 J Q K
0.76784533 
0.76045984 8 9 10 J Q
 5 : 农民2 : 9 10 J Q K | 3 4 4 5 8 8 9 A A 2 2 大王
0.17929573 
 6 :  地主 :  | 3 Q Q Q A A 2
0.8152164 
 7 : 农民1 :  | 3 3 4 5 5 6 6 7 7 10 J J K K K 2 小王
0.79557043 5
0.7921737 3
0.78308403 9
0.77781826 4 4
0.7627507 大王
 8 : 农民2 : 4 4 | 3 5 8 8 9 A A 2 2 大王
0.2037195 A A
0.18291381 
0.17791383 Q Q
 9 :  地主 :  | 3 Q Q Q A A 2
0.8274423 7 7
0.8267722 6 6
0.82449794 J J
0.8239323 
0.82265437 5 5
10 : 农民1 : 5 5 | 3 3 4 6 6 7 7 10 J J K K K 2

In [32]:
arena.winner

0

In [11]:
a1 = arena.records[0].copy(1, True)
a1.registerbot([bot_anneal.BOT(model,verbos=1, temperature=0), 
                   bot_anneal.BOT(model,verbos=1, temperature=0), 
                   bot_anneal.BOT(model,verbos=1, temperature=0)])
a1.wholegame()

0.48217815 7 7 8 8 9 9 10 10
0.44340256 7 7 8 8 9 9
0.43431252 8 8 9 9 10 10
0.3599209 3 4 5 6 7 8 9 10
0.35787514 3 4 5 6 7 8 9 10 J
 0 :  地主 : 7 7 8 8 9 9 10 10 | 3 4 5 6 6 J Q Q Q A A 2
0.53494 
 1 : 农民1 :  | 3 3 4 5 5 6 6 7 7 10 J J K K K 2 小王
0.57773286 
 2 : 农民2 :  | 3 4 4 5 8 8 9 9 10 J Q K A A 2 2 大王
0.48145744 3
0.45533037 5
0.3773461 J
0.3610569 2
0.36028445 A A
 3 :  地主 : 3 | 4 5 6 6 J Q Q Q A A 2
0.52965784 4
0.5283718 7
0.5282663 10
0.52798474 5
0.5274185 
 4 : 农民1 : 4 | 3 3 5 5 6 6 7 7 10 J J K K K 2 小王
0.47809815 5
0.45865792 10
0.45686048 J
0.45303148 Q
0.45116687 
 5 : 农民2 : 5 | 3 4 4 8 8 9 9 10 J Q K A A 2 2 大王
0.55159706 J
0.46630073 Q
0.4662622 2
0.46476704 6
0.46135175 A
 6 :  地主 : J | 4 5 6 6 Q Q Q A A 2
0.46248567 2
0.46246994 
0.462326 小王
0.46128535 K
 7 : 农民1 : 2 | 3 3 5 5 6 6 7 7 10 J J K K K 小王
0.39036041 
0.39033198 大王
 8 : 农民2 :  | 3 4 4 8 8 9 9 10 J Q K A A 2 2 大王
0.47907957 
 9 :  地主 :  | 4 5 6 6 Q Q Q A A 2
0.5404433 6 6
0.5404161 3 3
0.53814006 5
0.5371

In [11]:
a1.pos

0

In [12]:
a1.remain

array([[2, 2, 1, 4, 3, 1, 0, 2, 1, 1, 0, 1, 2, 0, 0],
       [1, 1, 1, 0, 0, 1, 1, 1, 2, 2, 2, 3, 1, 0, 1],
       [1, 1, 2, 0, 1, 2, 3, 1, 1, 1, 2, 0, 1, 1, 0]])

In [13]:
a1.lastplay

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [15]:
a1 = arena.records[0].copy(0, True)
a1.update(a1.getChoices()[0])